<a href="https://colab.research.google.com/github/andrew3690/F-A_root_project/blob/main/Research_Stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading Modules:
Spark, pyspark, Java.

Bellow it is the old fashion way, risky not Recomended

In [ ]:
# # Installing Java
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# # Dowload Spark
# !wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
# # Decompressing archives
# !tar xf spark-3.2.1-bin-hadoop2.7.tgz
# # installing findspark
# !pip install -q findspark
# # installing pyspark
# !pip install pyspark==3.2.1

In [ ]:
# Importing os library
# import os
# # Java envirorment variable
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# # Spark envirorment variable
# os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

# Better and consistent module that makes the same function as the code above

In [ ]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 46 kB/s 
     |████████████████████████████████| 200 kB 50.6 MB/s 
     |████████████████████████████████| 199 kB 9.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=154bf6c929503e74cc017237b2189e7683c490559d40da3200ecf3c21f8d9c7b
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


**Importing modules and pandas library that will be used later.**

In [ ]:
from pyspark.sql import SparkSession
import pyspark
import pandas as pd
# import findspark

# findspark.init()

In [ ]:
# instatiating spark instance
spark = SparkSession.builder.master('local[*]').getOrCreate()

spark

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


# Next cells acquire parquet data from remote files, and transpose it to spark format

In [ ]:
fille = "/content/gdrive/MyDrive/Data/Parquet/Formated/BrStocksFormated.parquet"

stocks = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

In [ ]:
fille = "/content/gdrive/MyDrive/Data/Parquet/Formated/Ebit.parquet"

ebit = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

In [ ]:
fille = "/content/gdrive/MyDrive/Data/Parquet/Formated/Stat.parquet"

stat = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

In [ ]:
fille = "/content/gdrive/MyDrive/Data/Parquet/Formated/Price.parquet"

price = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

Creating Tables in order to manipulate data in SQL format

In [ ]:
# Stocks table
stocks.createOrReplaceTempView("General")
# Ebit table
ebit.createOrReplaceTempView("Ebit")
# Stat table
stat.createOrReplaceTempView("Stat")
# Price table
price.createOrReplaceTempView("Price")

# Note 1: 
'DetToEquity' must be in percentage, try to get it done here or otherwise in Power Bi

---

# Note 2: 
Find a way to calculate Beta values from 
historical data get it from other sources, but try to calculate it localy (Talk with Felipe, about the period of time that will be used as 'ytd' or montlhy). 

# Fixed:
Beta values could be acquired trough yahoo finance

Listing brazilian companies that will be researched and getting hystorical financial data
filters that our Analyst indicates: Raking companies by: EBIT/EV > 0, ROE > 0, **Daily equity < 1.000.000.000**, revenueGrowth > 0 AND Will be done apart :(5-year-history) and all margins must be postive

In [ ]:
DFT = spark.sql("""
SELECT e.Ticker,
       Preco_Atual,
       Fluxo_de_caixa_total,
       Fluxo_de_caixa_por_acao,
       EBITDA,
       Divida_total,
       Liquidez_imediata,
       Liquidez_corrente,
       Receita_total,
       Divida_Patrimonio,
       Receita_por_acao,
       ROA,
       ROE,
       Lucro_Bruto,
       Fluxo_de_Caixa_Livre,
       Fluxo_de_caixa_operacional,
       Crescimento_de_Receita_3T,
       Margem_Bruta,
       Margem_EBITIDA,
       Margem_Operacional,
       Margem_liquida,
       Moeda,
       Crescimento_dos_ganhos_3T,
       EBIT,
       Divida_Liquida
FROM General g
LEFT JOIN Ebit e ON g.Ticker = e.Ticker
""").createOrReplaceTempView("Filtered")

In [ ]:
Stat = spark.sql("""
SELECT f.Ticker,
       Preco_Atual,
       Fluxo_de_caixa_total,
       Fluxo_de_caixa_por_acao,
       EBITDA,
       Divida_total,
       Liquidez_imediata,
       Liquidez_corrente,
       Receita_total,
       Divida_Patrimonio,
       Receita_por_acao,
       ROA,
       ROE,
       Lucro_Bruto,
       Fluxo_de_Caixa_Livre,
       Fluxo_de_caixa_operacional,
       Crescimento_de_Receita_3T,
       Margem_Bruta,
       Margem_EBITIDA,
       Margem_Operacional,
       Margem_liquida,
       Moeda,
       Crescimento_dos_ganhos_3T,
       EBIT,
       Divida_Liquida,
      Beta,
      Margem_de_Lucro,
      Crescimento_de_receitas_4T,
      Data_do_ultimo_dividendo,
      Valor_do_ultimo_dividendo
FROM Filtered f
LEFT JOIN Stat s ON f.Ticker = s.Ticker;
""").createOrReplaceTempView("Prstocks")

In [ ]:
DF = spark.sql("""
SELECT a.Ticker,
       Preco_Atual,
       Fluxo_de_caixa_total,
       Fluxo_de_caixa_por_acao,
       EBITDA,
       Divida_total,
       Liquidez_imediata,
       Liquidez_corrente,
       Receita_total,
       Divida_Patrimonio,
       Receita_por_acao,
       ROA,
       ROE,
       Lucro_Bruto,
       Fluxo_de_Caixa_Livre,
       Fluxo_de_caixa_operacional,
       Crescimento_de_Receita_3T,
       Margem_Bruta,
       Margem_EBITIDA,
       Margem_Operacional,
       Margem_liquida,
       Moeda,
       Crescimento_dos_ganhos_3T,
       EBIT,
       Divida_Liquida,
       Beta,
       Margem_de_Lucro,
       Crescimento_de_receitas_4T,
       Data_do_ultimo_dividendo,
       Valor_do_ultimo_dividendo,
       valor_de_mercado
FROM Prstocks a
LEFT JOIN Price p ON a.Ticker = p.Ticker;
""").createOrReplaceTempView('Stocks')

ev = número total de papeis x cotação + divida liquida
ev = market value +  liquid debt 

PSR = PREÇO DA AÇÃO / RECEITA LIQUIDA POR AÇÃO 

In [ ]:
CALCULATE = spark.sql("""
SELECT * FROM Stocks
WHERE Margem_Bruta > 0.0 AND Margem_EBITIDA > 0.0 AND 
Margem_Operacional > 0.0 AND Margem_liquida > 0.0 AND 
ROE > 0 AND Crescimento_de_Receita_3T > 0 AND 
((valor_de_mercado + Divida_Liquida)/EBIT) > 0.0;
""")

In [ ]:
df_final = CALCULATE.toPandas()

In [ ]:
df_final.to_excel("/content/gdrive/MyDrive/Data/XLXS/Final/StocksFinal.xlsx",
                  sheet_name="Stocks")